## Libraries

In [14]:
import pandas as pd
import numpy as np
import scipy.stats as stat

from math import sqrt
from mlgear.utils import show, display_columns
from surveyweights import normalize_weights
from survey_dud_detector import detect_straightlining


def margin_of_error(n=None, sd=None, p=None, type='proportion', interval_size=0.95):
    z_lookup = {0.8: 1.28, 0.85: 1.44, 0.9: 1.65, 0.95: 1.96, 0.99: 2.58}
    if interval_size not in z_lookup.keys():
        raise ValueError('{} not a valid `interval_size` - must be {}'.format(interval_size,
                                                                              ', '.join(list(z_lookup.keys()))))
    if type == 'proportion':
        se = sqrt(p * (1 - p)) / sqrt(n)
    elif type == 'continuous':
        se = sd / sqrt(n)
    else:
        raise ValueError('{} not a valid `type` - must be proportion or continuous')
    
    z = z_lookup[interval_size]
    return se * z


def print_pct(pct, digits=0):
    pct = pct * 100
    pct = np.round(pct, digits)
    if pct >= 100:
        if digits == 0:
            val = '>99.0%'
        else:
            val = '>99.'
            for d in range(digits - 1):
                val += '9'
            val += '9%'
    elif pct <= 0:
        if digits == 0:
            val = '<0.1%'
        else:
            val = '<0.'
            for d in range(digits - 1):
                val += '0'
            val += '1%'
    else:
        val = '{}%'.format(pct)
    return val


def calc_result(for_vote, against_vote, n, interval=0.8):
    GENERAL_POLLING_ERROR = 4.0
    ACQUIESENCE_BIAS = -5.0
    N_SIMS = 10000000
    
    for_moe = margin_of_error(n=n, p=for_vote/100, interval_size=interval)
    against_moe = margin_of_error(n=n, p=against_vote/100, interval_size=interval)
    undecided = 100 - for_vote - against_vote
    mean = for_vote + undecided * 0.25 + ACQUIESENCE_BIAS
    raw_moe = for_moe * 100 + against_moe * 100
    
    allocate_undecided = undecided * 0.4
    margin = raw_moe + allocate_undecided + GENERAL_POLLING_ERROR
    
    cdf_value = 0.5 + 0.5 * interval
    normed_sigma = stat.norm.ppf(cdf_value)
    sigma = margin / 100 / normed_sigma
    
    sims = np.random.normal(mean / 100, sigma, N_SIMS)
    chance_pass = np.sum([s > 0.5 for s in sims]) / N_SIMS
    low, high = np.percentile(sims, [20, 80]) * 100
    
    return {'mean': mean, 'high': high, 'low': low, 'n': n,
            'raw_moe': raw_moe, 'margin': margin, 'sigma': sigma, 'chance_pass': chance_pass}


def print_result(mean, high, low, n, raw_moe, margin, sigma, chance_pass):
    mean = np.round(mean, 1)
    first = np.round(high, 1)
    second = np.round(low, 1)
    sigma = np.round(sigma * 100, 1)
    raw_moe = np.round(raw_moe, 1)
    margin = np.round(margin, 1)
    chance_pass = print_pct(chance_pass, 1)
    if second < first:
        _ = first
        first = second
        second = _
    if second > 100:
        second = 100
    if first < 0:
        first = 0
    print(('Result {} (80% CI: {} to {}) (N={}) (raw_moe={}pts, margin={}pts, '
           'sigma={}pts) ({} likely to pass)').format(mean,
                                                      first,
                                                      second,
                                                      n,
                                                      raw_moe,
                                                      margin,
                                                      sigma,
                                                      chance_pass))
    print(('{} (80% CI: {} to {}) ({})').format(mean,
                                                first,
                                                second,
                                                chance_pass))
    print('-')

## Load Processed Data

In [15]:
survey = pd.read_csv('responses_processed_national_weighted.csv').fillna('Not presented')

## Insect Proposition

In [16]:
options = ['Vote for / Support', 'Vote against / Oppose', 'Don’t know / Undecided']

survey_ = survey.loc[survey['vote_measure_no_insect_feed'].isin(options)].copy()
survey_['weight'] = normalize_weights(survey_['weight'])
survey_['rv_weight'] = normalize_weights(survey_['rv_weight'])
survey_['lv_weight'] = normalize_weights(survey_['lv_weight'])

print('## vote_measure_no_insect_feed NATIONAL WEIGHTED ##')
lv_weighted_n = int(np.round(survey_['weight'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote_measure_no_insect_feed'].value_counts(normalize=True) * survey_.groupby('vote_measure_no_insect_feed')['weight'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
print_result(**calc_result(for_vote=votes['Vote for / Support'],
                           against_vote=votes['Vote against / Oppose'],
                           n=lv_weighted_n))

print('## vote_measure_no_insect_feed NATIONAL WEIGHTED + LV ##')
lv_weighted_n = int(np.round(survey_['lv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote_measure_no_insect_feed'].value_counts(normalize=True) * survey_.groupby('vote_measure_no_insect_feed')['lv_weight'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
print_result(**calc_result(for_vote=votes['Vote for / Support'],
                           against_vote=votes['Vote against / Oppose'],
                           n=lv_weighted_n))

## vote_measure_no_insect_feed NATIONAL WEIGHTED ##
Vote for / Support        27.200648
Vote against / Oppose     49.946462
Don’t know / Undecided    22.852890
dtype: float64
Result 27.9 (80% CI: 14.9 to 40.9) (N=334) (raw_moe=6.6pts, margin=19.8pts, sigma=15.4pts) (7.6% likely to pass)
27.9 (80% CI: 14.9 to 40.9) (7.6%)
-
## vote_measure_no_insect_feed NATIONAL WEIGHTED + LV ##
Vote for / Support        28.507542
Vote against / Oppose     48.907861
Don’t know / Undecided    22.584597
dtype: float64
Result 29.2 (80% CI: 16.0 to 42.2) (N=311) (raw_moe=6.9pts, margin=19.9pts, sigma=15.6pts) (9.0% likely to pass)
29.2 (80% CI: 16.0 to 42.2) (9.0%)
-


## Insect Feed Attitudes

In [17]:
questions = {'agree_flies_animal_feed': 'How much do you agree or disagree with the following?: Flies are a suitable source of protein for use in animal feed',
            'agree_eat_animals_fed_insects': 'How much do you agree or disagree with the following?: I would eat meat if I knew it the animal had been fed insects as part of its feed',
            'agree_eat_insects': 'How much do you agree or disagree with the following?: I would never personally eat insects',
            'agree_flies_animal_feed_alt': 'How much do you agree or disagree with the following?: It is acceptable to feed insects to animals that are raised for food',
            'agree_eat_animals_fed_insects_alt_reverse': 'How much do you agree or disagree with the following?: I would not be comfortable eating meat from an animal that was raised on insect feed',
            'agree_insect_feed_harms_health': 'How much do you agree or disagree with the following?: Eating meat from an animal that was raised on feed made from insects poses a risk to human health'}

options = ['Agree', 'Disagree', 'Don\'t know']

for var, question in questions.items():
    print('## {} ##'.format(question))
    survey_ = survey.loc[survey[var].isin(options)].copy()
    survey_['weight'] = normalize_weights(survey_['weight'])
    print(survey_[var].value_counts(normalize=True) * survey_.groupby(var)['weight'].mean() * 100)
    print('-')

## How much do you agree or disagree with the following?: Flies are a suitable source of protein for use in animal feed ##
Agree         24.288991
Disagree      34.214749
Don't know    41.496259
dtype: float64
-
## How much do you agree or disagree with the following?: I would eat meat if I knew it the animal had been fed insects as part of its feed ##
Agree         37.387022
Disagree      34.305828
Don't know    28.307151
dtype: float64
-
## How much do you agree or disagree with the following?: I would never personally eat insects ##
Agree         67.713651
Disagree      17.418804
Don't know    14.867545
dtype: float64
-
## How much do you agree or disagree with the following?: It is acceptable to feed insects to animals that are raised for food ##
Agree         37.297489
Disagree      30.490010
Don't know    32.212501
dtype: float64
-
## How much do you agree or disagree with the following?: I would not be comfortable eating meat from an animal that was raised on insect feed ##
Agre

In [18]:
straightlining_flies = detect_straightlining(survey[[c for c in survey.columns if 'agree_flies' in c or 'agree_eat' in c or 'agree_insects' in c]])
survey['meta_straightlining_flies'] = straightlining_flies
survey['meta_straightlining_flies'].value_counts()

0.6    1913
1.0    1107
0.4    1102
0.8     811
Name: meta_straightlining_flies, dtype: int64

In [21]:
pd.crosstab(survey['agree_flies_animal_feed'], survey['agree_flies_animal_feed_alt'])
# Flies are a suitable source of protein for use in animal feed
# It is acceptable to feed insects to animals that are raised for food

agree_flies_animal_feed_alt,Agree,Disagree,Don't know,Not presented
agree_flies_animal_feed,,,,
Agree,857,42,89,0
Disagree,181,660,251,0
Don't know,651,229,1115,0
Not presented,0,0,0,858


In [22]:
pd.crosstab(survey['agree_eat_animals_fed_insects'], survey['agree_eat_animals_fed_insects_alt_reverse'])
# I would eat meat if I knew it the animal had been fed insects as part of its feed
# I would not be comfortable eating meat from an animal that was raised on insect feed

agree_eat_animals_fed_insects_alt_reverse,Agree,Disagree,Don't know,Not presented
agree_eat_animals_fed_insects,,,,
Agree,246,1272,226,0
Disagree,847,168,118,0
Don't know,244,232,722,0
Not presented,0,0,0,858


In [24]:
survey.loc[:, 'flies_comprehension'] = 1
survey.loc[survey['agree_flies_animal_feed'] == 'Don\'t know', 'flies_comprehension'] = -1
survey.loc[survey['agree_flies_animal_feed'] == 'Not presented', 'flies_comprehension'] = -2
survey.loc[survey['agree_flies_animal_feed_alt'] == 'Don\'t know', 'flies_comprehension'] = 0
survey.loc[survey['agree_flies_animal_feed_alt'] == 'Not presented', 'flies_comprehension'] = -2
survey.loc[(survey['agree_flies_animal_feed'] == 'Agree') & (survey['agree_flies_animal_feed_alt'] == 'Disagree'), 'flies_comprehension'] = 0
survey.loc[(survey['agree_flies_animal_feed'] == 'Disgree') & (survey['agree_flies_animal_feed_alt'] == 'Agree'), 'flies_comprehension'] = 0

survey.loc[survey['agree_eat_animals_fed_insects'] == 'Don\'t know', 'flies_comprehension'] = -1
survey.loc[survey['agree_eat_animals_fed_insects'] == 'Not presented', 'flies_comprehension'] = -2
survey.loc[survey['agree_eat_animals_fed_insects_alt_reverse'] == 'Don\'t know', 'flies_comprehension'] = -1
survey.loc[survey['agree_eat_animals_fed_insects_alt_reverse'] == 'Not presented', 'flies_comprehension'] = -2
survey.loc[(survey['flies_comprehension'] >= 0) & (survey['agree_eat_animals_fed_insects'] == 'Agree') & (survey['agree_eat_animals_fed_insects_alt_reverse'] == 'Agree'), 'flies_comprehension'] = 0
survey.loc[(survey['flies_comprehension'] >= 0) & (survey['agree_eat_animals_fed_insects'] == 'Disagree') & (survey['agree_eat_animals_fed_insects_alt_reverse'] == 'Disagree'), 'flies_comprehension'] = 0

survey['flies_comprehension'].value_counts()

-1    2143
 1    1127
-2     858
 0     805
Name: flies_comprehension, dtype: int64

In [25]:
pd.crosstab(survey['flies_comprehension'], survey['meta_straightlining_flies'])

meta_straightlining_flies,0.4,0.6,0.8,1.0
flies_comprehension,,,,
-2,0,0,0,858
-1,689,843,469,142
0,393,201,104,107
1,20,869,238,0


## Animal Pain Attitudes

In [19]:
options = ['Agree', 'Disagree', 'Don\'t know']

for c in survey.columns:
    if 'agree_pain' in c:
        print('## {} ##'.format(c))
        survey_ = survey.loc[survey[var].isin(options)].copy()
        survey_['weight'] = normalize_weights(survey_['weight'])
        print(survey_[c].value_counts(normalize=True) * survey_.groupby(c)['weight'].mean() * 100)
        print('-')

## agree_pain_dogs ##
Agree         96.519123
Disagree       1.964506
Don't know     1.516371
dtype: float64
-
## agree_pain_pigs ##
Agree         93.419720
Disagree       2.701108
Don't know     3.879172
dtype: float64
-
## agree_pain_chickens ##
Agree         92.944797
Disagree       2.766880
Don't know     4.288323
dtype: float64
-
## agree_pain_fish ##
Agree         77.687991
Disagree       6.577931
Don't know    15.734078
dtype: float64
-
## agree_pain_honeybees ##
Agree         65.144263
Disagree       7.923408
Don't know    26.932329
dtype: float64
-
## agree_pain_termites ##
Agree         51.618358
Disagree      13.051448
Don't know    35.330194
dtype: float64
-
## agree_pain_ants ##
Agree         55.932779
Disagree      11.913711
Don't know    32.153509
dtype: float64
-


In [20]:
straightlining_pain = detect_straightlining(survey[[c for c in survey.columns if 'agree_pain' in c]])
straightlining_pain.value_counts()

1.000000    2926
0.571429    1181
0.714286     479
0.857143     189
0.428571     158
dtype: int64

In [27]:
pd.crosstab(survey['agree_pain_ants'], survey['agree_flies_animal_feed'], normalize='index')

agree_flies_animal_feed,Agree,Disagree,Don't know,Not presented
agree_pain_ants,,,,
Agree,0.257245,0.283549,0.459206,0.0
Disagree,0.290672,0.284165,0.425163,0.0
Don't know,0.202042,0.237053,0.560904,0.0
Not presented,0.000000,0.000000,0.000000,1.0
